In [93]:
import pandas as pd
import pyarrow.parquet as pq

In [121]:
# Cargar las tablas desde los archivos parquet
steam_games = pq.read_table("steam_games.parquet").to_pandas()
users_items = pq.read_table("users_items.parquet").to_pandas()
user_reviews = pq.read_table("user_reviews.parquet").to_pandas()

In [109]:
def PlayTimeGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="left")
    
    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]
    
    # Convertir la columna "release_date" a tipo datetime
    try:
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)
    except ValueError:
        # Si la inferencia de formato falla, intentar especificar el formato manualmente
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='coerce', format='%b %Y')

    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].dt.year

    # Agrupar por año y calcular la suma de las horas jugadas para cada año
    result = genre_data.groupby('year')['playtime_forever'].sum().reset_index()

    # Encontrar el año con la mayor cantidad de horas jugadas
    max_year = result.loc[result['playtime_forever'].idxmax()]['year']

    return max_year

In [110]:
# Ejemplo de uso
genre = "Action"  # Reemplaza con el género deseado
result = PlayTimeGenre(genre)
print(f"El año con más horas jugadas para el género {genre} es: {result}")

/var/folders/7l/jr096cm117gfqxv28vmgt9b80000gp/T/ipykernel_67711/37360139.py:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)


El año con más horas jugadas para el género Action es: 2015.0


In [278]:
def UserForGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="left")
    print("MERGED DATA SHAPE")
    print(merged_data.shape)
    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    print("MERGED DATA SIN VACIOS SHAPE")
    print(merged_data.shape)
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]
    print("GENRE DATA SOLO GENERO SHAPE")
    genre_data.to_csv('genre_data.csv', index=False)
    print(genre_data.shape)
    
    # Convertir la columna "release_date" a tipo datetime
    try:
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)
    except ValueError:
        # Si la inferencia de formato falla, intentar especificar el formato manualmente
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='coerce', format='%b %Y')

    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].dt.year
    genre_data['playtime_forever'] = pd.to_numeric(genre_data['playtime_forever'], errors='coerce')
    genre_data = genre_data.dropna(subset=['year', 'user_id', 'playtime_forever'])
    print("GENRE DATA SIN NULOS SHAPE")
    print(genre_data.shape)
    
    # Agrupar por usuario y calcular la suma de las horas jugadas para cada usuario
    user_total_playtime = genre_data.groupby('user_id')['playtime_forever'].sum()
    user_total_playtime = user_total_playtime.reset_index()

    # limpiar datos invalidos
    #user_total_playtime['playtime_forever'] = pd.to_numeric(user_total_playtime['playtime_forever'], errors='coerce')
    #user_total_playtime = user_total_playtime.dropna()
    
    # Obtener el usuario con más horas jugadas
    sorted_users = user_total_playtime.sort_values(by='playtime_forever', ascending=False)
    sorted_users = sorted_users.reset_index()
    sorted_users.drop('index', axis=1, inplace=True)
    
    # Obtener el usuario con más horas jugadas
    max_user = sorted_users['user_id'].iloc[0]
    
    # Filtrar los datos solo para el usuario con la máxima cantidad de horas jugadas
    max_user_data = genre_data[genre_data['user_id'] == max_user]

    # Agrupar por año y calcular la suma de las horas jugadas para ese usuario
    year_playtime = max_user_data.groupby('year')['playtime_forever'].sum().reset_index()
    
    # Convertir a lista de tuplas (año, horas acumuladas)
    year_playtime_list = list(year_playtime.to_records(index=False))

    return max_user, year_playtime_list

In [279]:
# Ejemplo de uso
genre = "Casual"  # Reemplaza con el género deseado
UserForGenre(genre)

MERGED DATA SHAPE
(5153477, 21)
MERGED DATA SIN VACIOS SHAPE
(5153477, 21)
GENRE DATA SOLO GENERO SHAPE
(500418, 21)
GENRE DATA SIN NULOS SHAPE
(72, 22)


/var/folders/7l/jr096cm117gfqxv28vmgt9b80000gp/T/ipykernel_67711/3094917327.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)


('76561198089944936', [(2014., 18533.)])